In [10]:
from gensim.models import Word2Vec
import nltk
import glob
import os
import string
import numpy
import re
import gensim
import random
from tqdm import tqdm
from gensim.models import KeyedVectors
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

In [11]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
tt = pd.read_csv('tokens_year.csv')

In [12]:
os.chdir("D://Scriptie//Data//models")   #//10yp_s150
list_mod = glob.glob('*w2v') 
list_mod

['1815_10yp_bigrams_model.w2v',
 '1825_10yp_bigrams_model.w2v',
 '1835_10yp_bigrams_model.w2v',
 '1845_10yp_bigrams_model.w2v',
 '1855_10yp_bigrams_model.w2v',
 '1865_10yp_bigrams_model.w2v',
 '1875_10yp_bigrams_model.w2v',
 '1885_10yp_bigrams_model.w2v',
 '1895_10yp_bigrams_model.w2v',
 '1905_10yp_bigrams_model.w2v']

In [13]:
def GetNetwork(word, year):
    mfn = [m for m in list_mod if int(m[0:4]) == year]
    
    #model = KeyedVectors.load_word2vec_format(mfn[0], binary = True)
    model = KeyedVectors.load(mfn[0], mmap='r')
    
    
    ttp = tt[(tt.year >= year) & (tt.year < (year + 10))]
    ttp = ttp['tokens'].sum()
    
    dft = pd.DataFrame()
    
    for wl1 in list(dict(model.most_similar(word, topn=8)).keys()):

        for wl2 in list(dict(model.most_similar(wl1, topn=8)).keys()):
            
            for wl3 in list(dict(model.most_similar(wl2, topn=8)).keys()):
                
                for wl4 in list(dict(model.most_similar(wl3, topn=8)).keys()):
                    source = wl3
                    target = wl4
                    t = pd.DataFrame([source, target]).T
                    dft = dft.append(t)
    
    dft.columns = ['source', 'target']
    dft['count'] = [model.wv.vocab[w].count for w in list(dft.target)]
    dft['count'] = dft['count'] / ttp
    
    
    #dgc = Counter(list(dft.target))
    #dgc = [k for k,v in dict(dgc).items() if v >= 100]
    
    dft['degree'] = 1
    
    #dft = dft[dft.target.isin(dgc)]
    
    return dft

In [14]:
d = GetNetwork("het_buitenland", 1905)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: Deprec

In [6]:
d['degree'] = d.groupby('target')['target'].transform('count')

In [7]:
d = d[d['degree'] > 10]

In [15]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//Visualizations//Poster")
d.to_csv('network_het_buitenland.csv', index=False)

In [47]:
op = pd.DataFrame()

for i in range(1825,1906,10):
    d = GetNetwork("buitenlanders", i)
    dc = Counter(d['target']).most_common(12)
    list_values = [round(v/len(d), 3) for k,v in dc]
    list_values = [" (" + str(v) + ")" for v in list_values]
    list_values = list(zip([k for k,v in dc], list_values))
    t = pd.DataFrame([" ".join([k,v]) for k,v in list_values])
    t.columns = [str(i)]
    df_c = pd.concat([op.reset_index(drop=True), t], axis=1)
    op = df_c


C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [70]:
op = pd.DataFrame()

for i in range(1815,1906,20):
    mfn = [m for m in list_mod if int(m[0:4]) == i]
    m = KeyedVectors.load(mfn[0], mmap='r')
    #m = KeyedVectors.load_word2vec_format(mfn[0], binary = True)
    t = pd.DataFrame(list(dict(m.most_similar("werelddeel", topn=15))))
    t.columns = [str(i)]
    df_c = pd.concat([op.reset_index(drop=True), t], axis=1)
    op = df_c

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [75]:
m.most_similar('wereldpolitiek',topn= 50)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('veroveringspolitiek', 0.6817159652709961),
 ('politieke_macht', 0.6632646322250366),
 ('wereldheerschappij', 0.6603294610977173),
 ('hegemonie', 0.6540054082870483),
 ('staatkundo', 0.6362661123275757),
 ('europesche_staatkunde', 0.6300525665283203),
 ('eilandenrijk', 0.6280711889266968),
 ('politieke_verhoudingen', 0.6139512062072754),
 ('russificatie', 0.6111616492271423),
 ('duitsche_politiek', 0.6099926829338074),
 ('historische_ontwikkeling', 0.6094818115234375),
 ('veroveringszucht', 0.6083558797836304),
 ('natuurlijke_ontwikkeling', 0.6051191687583923),
 ('politieke_geschiedenis', 0.6029306650161743),
 ('russische_politiek', 0.602576494216919),
 ('regeeringsstelsel', 0.5984328985214233),
 ('balkanquaestie', 0.5967118740081787),
 ('oostenrijksche_politiek', 0.5966980457305908),
 ('invloedsfeer', 0.5952433347702026),
 ('militaire_mogendheid', 0.5948401689529419),
 ('overheersching', 0.5922002792358398),
 ('staatswezen', 0.591489851474762),
 ('volksregering', 0.591326892375946),

In [63]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//Visualizations//CH3//source-files")
op.to_csv('most_similar_terms_wereld.csv', index=False)
os.chdir("D://Scriptie//Data//models//10yp_s150")


In [35]:
m

NameError: name 'm' is not defined

In [33]:
dft = pd.DataFrame()

for i in range(1815,1906,10):
    df = GetNetwork("werelddeel", i)
    l = list(dict(Counter(list(df['target'])).most_common(20)).keys())
    print(" | ".join(l))
    df = pd.DataFrame(l)
    df['year'] = i
    dft = dft.append(df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: Deprec

schiereiland | fchiereiland | vasteland | iand | schier_eiland | lsnd | land | vaste_land | laad | geheele_land | liet_eiland | voorregt | bet_eiland | denkbeeld | noodlot | hee_eiland | het_eiland | hec_eiland | kordon | lard
gemeenebest | bondgenootschap | keizerrijk | laud | koningrijk | schiereiland | iand | land | lsnd | middelpunt | rijk | werelddeel | verbond | vorstendom | koninkrijk | huisje | lard | bondgenootfchap | regeringstelsel | kerkhof
keizerrijk | koningrijk | schiereiland | laud | rgk | hertogdom | vasteland | jand | fchiereiland | koninkrijk | vorstendom | gemeenebest | bondgenootschap | steunpunt | land | werelddeel | schier_eiland | russische_rijk | middenpunt | koningrijk_polen
schiereiland | vasteland | jand | keizerrijk | iand | rgk | schier_eiland | laud | fchiereiland | lsnd | koningrijk | element | land | gemeenebest | eiland | middenpunt | steunpunt | middelpunt | hertogdom | vorstendom
schiereiland | ryk | vasteland | fchiereiland | jand | overige_europa |

In [7]:
os.chdir('C://Users//Ruben//Desktop')
d.to_csv('kwestie-1875.csv', index=False)

In [35]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//Visualizations//CH2//source-files")
dft.to_csv('werelddeel_mct.csv', index=False)